In [1]:
pip install keras-self-attention

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lime

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import re
import nltk
import csv
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
style.use('ggplot')
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from keras_self_attention import SeqSelfAttention
import lime
from lime import lime_text
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [4]:
# Import LIME for text explanations
from lime.lime_text import LimeTextExplainer

In [5]:
raw_data = pd.read_csv('PROMISE_exp1.csv')

In [6]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

True

In [7]:
# Define preprocessing function
def preprocess_text(text):
  # Remove special characters and digits
  text = re.sub(r'[^a-zA-Z]', ' ', text)
  # Convert to lowercase
  text = text.lower()
  # Tokenize the text
  tokens = word_tokenize(text)
  # Remove stopwords
  tokens = [word for word in tokens if word not in stop_words]
  # Join the tokens back into a single string
  preprocessed_text = ' '.join(tokens)
  preprocessed_text = text.lower() # Example: Convert text to lowercase

  return preprocessed_text


In [8]:
import nltk
nltk.download('stopwords')


# Define stop_words
stop_words = set(nltk.corpus.stopwords.words('english'))



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Apply preprocessing to your text data
raw_data['RequirementText'] = raw_data['RequirementText'].apply(preprocess_text)

In [10]:
# Split your data into features (X) and labels (y)
X = raw_data['RequirementText']
y = raw_data['_class_']


In [11]:
# Encode your labels if they are not already encoded
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


In [12]:
# Define tokenization and padding parameters
max_words = 1000 # Maximum number of words to keep based on word frequency
max_len = 100 # Maximum sequence length


# Tokenize your text data
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_seq, maxlen=max_len)


In [13]:
# Define and compile your LSTM model
model = Sequential()
model.add(Embedding(max_words, 64, input_length=max_len))
model.add(LSTM(64, return_sequences=True))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [14]:
# Define number of cross-validation folds
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=3)

In [15]:
# Initialize a list to store cross-validation accuracies
accuracies = []

In [16]:
# Perform cross-validation
for train_index, test_index in skf.split(X_padded, y):
  X_train, X_test = X_padded[train_index], X_padded[test_index]
  y_train, y_test = y[train_index], y[test_index]

In [17]:
# Train your model on the training data
model.fit(X_train, y_train.astype('float'), epochs=10, batch_size=32, verbose=0)

In [18]:
# Make predictions on the test data
y_pred = (model.predict(X_test) >= 0.5).astype('int')


7/7 [==============================] - 1s 58ms/step


In [19]:
# Calculate accuracy and store it
accuracy = accuracy_score(y_test, y_pred)
accuracies.append(accuracy)

In [20]:
# Split your data into training and testing sets for final model training
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=3)


In [21]:
# Train your final model on the training data
model.fit(X_train, y_train.astype('float'), epochs=10, batch_size=32, verbose=0)


In [22]:
# Make predictions on the training and testing data
y_pred_train = (model.predict(X_train) >= 0.5).astype('int')
train_accuracy = accuracy_score(y_train.astype('int'), y_pred_train)

y_pred_test = (model.predict(X_test) >= 0.5).astype('int')
test_accuracy = accuracy_score(y_test.astype('int'), y_pred_test)


7/7 [==============================] - 0s 62ms/step


In [23]:
# Print confusion matrices and classification reports
print("Test Confusion Matrix:")
print(confusion_matrix(y_test.astype('int'), y_pred_test))
print("\nTest Classification Report:")
print(classification_report(y_test.astype('int'), y_pred_test))

print("Train Confusion Matrix:")
print(confusion_matrix(y_train.astype('int'), y_pred_train))
print("\nTrain Classification Report:")
print(classification_report(y_train.astype('int'), y_pred_train))



Test Confusion Matrix:
[[ 80  12]
 [  0 102]]

Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.87      0.93        92
           1       0.89      1.00      0.94       102

    accuracy                           0.94       194
   macro avg       0.95      0.93      0.94       194
weighted avg       0.94      0.94      0.94       194

Train Confusion Matrix:
[[343   9]
 [  3 420]]

Train Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       352
           1       0.98      0.99      0.99       423

    accuracy                           0.98       775
   macro avg       0.99      0.98      0.98       775
weighted avg       0.98      0.98      0.98       775

Test Confusion Matrix:
[[ 80  12]
 [  0 102]]

Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.87      0.93        92
          

In [24]:
# Initialize LIME Text Explainer
explainer = LimeTextExplainer(class_names=["F", "NF"])

In [25]:
import sys
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from lime.lime_text import LimeTextExplainer
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QTextBrowser, QFileDialog


In [26]:
# Define a predict function that takes a list of texts as input
def predict_function(texts):
    # Preprocess the input text
    preprocessed_texts = [preprocess_text(text) for text in texts]

    # Tokenize and pad the preprocessed texts (assuming you have tokenizer and max_len defined)
    X_seq = tokenizer.texts_to_sequences(preprocessed_texts)
    X_padded = pad_sequences(X_seq, maxlen=max_len)

    # Make predictions using your model (assuming model is defined)
    predictions = model.predict(X_padded)

    # Assuming your model returns a single label for each input, convert it to a probability distribution
    # If your model returns probabilities directly, you can skip this step
    predictions = np.column_stack([1 - predictions, predictions])  # Assuming binary classification

    return predictions

In [27]:
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QTextBrowser, QFileDialog, QVBoxLayout, QWidget
from PyQt5.QtGui import QFont, QIcon
from PyQt5.QtCore import QSize
# Add this import at the top of your code
from PyQt5.QtGui import QFont
from PyQt5.QtWidgets import QVBoxLayout, QWidget

In [28]:
num_samples = 1000
class ExplanationApp(QMainWindow):
    def __init__(self):
        super().__init__()

        # Initialize LIME Text Explainer
        self.explainer = LimeTextExplainer(class_names=["F", "NF"])

        # Create a button to load and explain requirements
        self.explain_button = QPushButton('Explain Requirements', self)
        self.explain_button.clicked.connect(self.explain_requirements)
        self.explain_button.setGeometry(10, 10, 180, 30)
        
        
        # Set the CSS style of the button to change the hover effect to blue
        self.explain_button.setStyleSheet("""
            QPushButton:hover {
                background-color: blue;
            }
        """)
        
        # Create a text browser to display the explanations
        self.result_text = QTextBrowser(self)
        self.result_text.setGeometry(10, 50, 780, 540)

        self.setWindowTitle('Requirement Explanation App')
        self.setGeometry(100, 100, 800, 600)

    def explain_requirements(self):
        # Open a file dialog to select the input text file
        file_dialog = QFileDialog(self)
        file_dialog.setFileMode(QFileDialog.ExistingFile)
        file_dialog.setNameFilter("Text files (*.txt)")
        file_dialog.setViewMode(QFileDialog.List)
        file_dialog.setOption(QFileDialog.ReadOnly, True)
        file_dialog.setLabelText(QFileDialog.Accept, "Load")
        file_dialog.setWindowTitle("Select a text file")
        file_path, _ = file_dialog.getOpenFileName()

        if not file_path:
            return

        # Read and preprocess requirements from the selected file
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        explanations = []

        for i, line in enumerate(lines):
            preprocessed_text = preprocess_text(line)
            preprocessed_text = preprocessed_text.lower()
            preprocessed_text = ' '.join([word for word in preprocessed_text.split() if word not in stop_words and len(word) > 2])

            # Get predicted probabilities using your predict_function
            predicted_probabilities = predict_function([preprocessed_text])

            # Get the predicted class label
            predicted_label = label_encoder.classes_[np.argmax(predicted_probabilities)]

            # Explain the prediction for the current requirement
            explanation = self.explainer.explain_instance(preprocessed_text, predict_function, num_features=10, num_samples=num_samples)

            explanations.append(f"Requirement {i + 1} Text: {line}\n")
            explanations.append(f"Predicted Class Label: {predicted_label}\n")
            explanations.append(f"Explanation:\n{explanation.as_list()}\n")
            explanations.append("=" * 50)

        self.setStyleSheet(
    """
    QMainWindow {
        background-color: #f2f2f2;
    }

    QPushButton {
        background-color: #007BFF;
        color: white;
        border: none;
        border-radius: 5px;
    }

    QPushButton:hover {
                background-color: blue;
            }

    QTextBrowser {
        background-color: white;
        color: black;
        border: 1px solid #ccc;
        border-radius: 5px;
        font-family: Arial;
        font-size: 12pt;
    }
    """
)

        # Example of adding an icon to a button
        self.explain_button.setIcon(QIcon("icon.png"))
        
        # Create a central widget
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)

        # Create a vertical layout for the central widget
        layout = QVBoxLayout()
        layout.addWidget(self.explain_button)
        layout.addWidget(self.result_text)
        central_widget.setLayout(layout)


        # Display the explanations in the text browser
        self.result_text.clear()
        self.result_text.setPlainText("".join(explanations))


In [29]:
def main():
    app = QApplication(sys.argv)
    ex_app = ExplanationApp()
    ex_app.show()
    sys.exit(app.exec_())

In [30]:
if __name__ == '__main__':
    main()

32/32 [==============================] - 2s 61ms/step


SystemExit: 0

SystemExit: 0

C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
